# Transfer Learning in Keras

In this notebook, we load a pre-trained model (in this case, VGGNet19) and finetune it for a new task: detecting hot dogs.

#### Load dependencies

In [1]:
from tensorflow.keras.applications.vgg19 import VGG19 # new!
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator # new!

#### Load the pre-trained VGG19 model

In [2]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

80142336/80134624 [==============================] - 2s 0us/step


#### Freeze all the layers in the base VGGNet19 model

In [3]:
for layer in vgg19.layers:
    layer.trainable = False

#### Add custom classification layers

In [4]:
# Instantiate the sequential model and add the VGG19 model: 
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model: 
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))

#### Compile the model for training

In [5]:
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

#### Prepare the data for training

The dataset is available for download [here](https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog/home). You should download the zipfile and extract the contents into a folder called `hot-dog-not-hot-dog` in the `notebooks` directory.

In [6]:
# Instantiate two image generator classes:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [7]:
# Define the batch size:
batch_size=32

In [8]:
# Define the train and validation generators: 
train_generator = train_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [9]:
model.fit_generator(train_generator, steps_per_epoch=15, 
                    epochs=16, validation_data=valid_generator, 
                    validation_steps=15)

Epoch 1/16


15/15 [==============================] - 71s 5s/step - loss: 1.3114 - accuracy: 0.5386 - val_loss: 0.6006 - val_accuracy: 0.6792
Epoch 2/16
15/15 [==============================] - 70s 5s/step - loss: 0.9260 - accuracy: 0.6416 - val_loss: 0.8028 - val_accuracy: 0.6500
Epoch 3/16
15/15 [==============================] - 70s 5s/step - loss: 0.6833 - accuracy: 0.6996 - val_loss: 1.0217 - val_accuracy: 0.5917
Epoch 4/16
15/15 [==============================] - 71s 5s/step - loss: 0.6606 - accuracy: 0.7354 - val_loss: 0.9002 - val_accuracy: 0.6375
Epoch 5/16
15/15 [==============================] - 70s 5s/step - loss: 0.6189 - accuracy: 0.7339 - val_loss: 0.5459 - val_accuracy: 0.7604
Epoch 6/16
15/15 [==============================] - 70s 5s/step - loss: 0.4894 - accuracy: 0.7876 - val_loss: 0.4412 - val_accuracy: 0.8021
Epoch 7/16
15/15 [==============================] - 70s 5s/step - loss: 0.4099 - accuracy: 0.8240 - val_loss: 0.6143 - val_accuracy: 0.7333
Epoch 8/16
15/15 [=============